<h1>The Beginning: Slice Plots!</h1>
First, basic loading of data, then lets start by making a slice plot!

In [ ]:
import yt
ds = yt.load('/Users/jillnaiman1/data/IsolatedGalaxy/galaxy0030/galaxy0030')

Here we will start with a slice along the x-axis of our sim with the center at the maximum of the variable we are plotting - density

In [ ]:
s = yt.SlicePlot(ds, "x", ["density"], center="max")

Now, lets see our default plot, around the max

In [ ]:
s.show()

We can also center around other things, like the actual center of the domain:

In [ ]:
s = yt.SlicePlot(ds, "x", ["density"], center="max")
s.show()

Given that the center of our sim isn't that different from the center of the maximum density, these plots basically look the same.

Other center arguments can be like ("min", field) or ("max", field) where field is one of the variables in your data set or a derived variable.

Finally, you can also set the center by hand:

In [ ]:
s = yt.SlicePlot(ds, "x", ["density"], center=[0.55, 0.5, 0.5])
s.show()

The above is now a slice plot that instead of being at the center of the data set is a little further out along the x-axis so you are looking at the larger scale structure outside the centeral over density.

Ok, now lets start moving out our data cube with slice plots.  I think we might wanna zoom in a bit, so lets do that!

In [ ]:
s = yt.SlicePlot(ds, "x", ["density"], center="max")
s.zoom(2.5)
s.show()

But you know what now? lets try annotating this thing!  Lets start with some temperature contours.

In [ ]:
s.annotate_contour("temperature")


You know what? Lets change the color map from the default.

In [ ]:
s.set_cmap('density','jet')
s.show()

Finally, lets maybe put on some velocity vectors!

In [ ]:
s.annotate_velocity()

Like it? then you can keep it!

In [ ]:
s.save('/Users/jillnaiman1/Desktop/myslice.png')

But lets say you wanna clear everything and just plot some grids

In [ ]:
s.annotate_clear()
s.show()

Now, lets over plot some grids for fun!

In [ ]:
s.annotate_grids()
s.show()

Lets zoom in and plot the actual cell edges

In [ ]:
s.set_width((20, 'kpc'))
s.annotate_cell_edges()
s.show()

Finally, lets say something silly:

In [ ]:
s.annotate_clear()
s.annotate_title('I am the most awesomest plot')
s.annotate_text((5, 5), 'Galaxy!', coord_system='plot')

If you wanna make a really pretty slice plot that isn't distracted by all that mathy axis and color bar stuff, you are also welcome to totally do that:

In [ ]:
s_pretty = yt.SlicePlot(ds, 'z', 'density', width=(10,'kpc'))
s_pretty.hide_colorbar()
s_pretty.hide_axes()
s_pretty.show()


You can also change a bit how the axis are displayed if you choose to still keep them in your plot:

In [ ]:
s_fonts = yt.SlicePlot(ds, 'z', 'density', width=(10,'kpc'))
s_fonts.set_font({'family': 'sans-serif', 'style': 'italic',
              'weight': 'bold', 'size': 24})
s_fonts.show()


More options for annotations are available on: http://yt-project.org/doc/visualizing/callbacks.html#annotate-clear

Also, if you want to further modify your plots on a more nitty gritty level (including directly throught matplotlib), check out more details on: http://yt-project.org/docs/dev/visualizing/plots.html

<h1>Now Moar plots: Projection, Phase, Profile!</h1>
Lets check out some of the other neato keen plots you can make simply with yt!

<h3>(1) Projections</h3>

In [ ]:
p = yt.ProjectionPlot(ds, "y", "density")
p.show()

The above is an example of an unweighted projection plot: this means that the plotted variable is just integrated along the line of sight g(x) = int(f(x,y,z)*ds), where ds is the unit vector along the line of sight.  But! We can also weight by stuff:

In [ ]:
pw = yt.ProjectionPlot(ds, "x", "temperature", weight_field="density")
pw.show()

The above is like g(x) = int(f(x,y,z)*w(x,y,z)*ds)/int(w(x,y,z)*ds).  Other options are "mip" which gives the max along the axis, or "sum" which does a direct summation (useful for example when 3rd axis is like frequency or something).

<h4>Off Axis!</h4>
Before moving onto some more analysis-y type plots, one thing worth mentioning is that you can make both slice plots and projection plots off axis - so... that's pretty sweet.  Lets do some!

In principle, you can give it any old vector with which to do the rotation about.  For fun, lets project everything along the rotation vector!


In [ ]:
# Create a 15 kpc radius sphere, centered on the center of the sim volume
# in this way, we can do our projection over a subset of the domain
sp = ds.sphere("center", (15.0, "kpc"))

# Get the angular momentum vector for the sphere.
L = sp.quantities.angular_momentum_vector()

print("Angular momentum vector: {0}".format(L))

In [ ]:
soff = yt.OffAxisSlicePlot(ds, L, "density", sp.center, (25, "kpc"))
soff.show()

Lets also do an off axis projection plot!

In [ ]:
poff = yt.OffAxisProjectionPlot(ds, L, "density", sp.center, (25, "kpc"))
poff.show()

<h3>(2) Phase Plots</h3>
Now maybe you wanna know something about the content of your sim - like how much cold dense gas is there in your sim?  To do this we can make a phase plot.

Phase Plots are best thought of as plotting a distribution of points, either taking the average or the accumulation in a bin. The default behavior is to average, using the cell mass as the weighting, but this behavior can be controlled through the weight_field parameter.

In [ ]:
my_sphere = ds.sphere("c", (50, "kpc"))
ph = yt.PhasePlot(my_sphere, "density", "temperature", ["H_fraction"],
                        weight_field=None)
ph.show()

If you would rather see the average value of a field as a function of two other fields, leave off the weight_field argument, and it will average by the cell mass. Like so:

In [ ]:
ph_cellmass = yt.PhasePlot(my_sphere, "density", "temperature", ["H_fraction"])
ph_cellmass.show()

<h3>(3) Simple Profile plots</h3>
Now maybe you wanna take it down a notch and look at your data in different sorts of 1D plots... lets do it!

For example, lets look at how the temperture changes as a function of density within the spiral galaxy part of our IsolatedGalaxy simulation.  This will create a YTDisk centered at [0.5, 0.5, 0.5], with a normal vector of [0.0, 0.0, 1.0], radius of 10 kiloparsecs and height of 3 kiloparsecs and will then make a plot of the mass-weighted average temperature as a function of density for all of the gas contained in the cylinder.

In [ ]:
my_galaxy = ds.disk(ds.domain_center, [0.0, 0.0, 1.0], 10*yt.units.kpc, 3*yt.units.kpc)
p1d = yt.ProfilePlot(my_galaxy, "density", ["temperature"])
p1d.show()

We might also want to know how this looks over a whole sphere.  Maybe how much gas is at a certain temperature.

In [ ]:
my_sphere = ds.sphere([0.5, 0.5, 0.5], (100, "kpc"))
p1d_sp = yt.ProfilePlot(my_sphere, "temperature", ["cell_mass"],
                      weight_field=None)
p1d_sp.show()

Note that because we have specified the weighting field to be None, the profile plot will display the accumulated cell mass as a function of temperature rather than the average.  But we could totally weight by something like density.

In [ ]:
p1d_spw = yt.ProfilePlot(my_sphere, "temperature", ["cell_mass"],
                      weight_field='density')
p1d_spw.show()

So we see there is more dense stuff at lower temperatures... I think this is what this says.

Also, sometimes maybe you wanna see how something accumulates over radius.  This happens a lot with mass - we want to know how much mass is in a sphere.  We can do this like:

In [ ]:
p1d_acc = yt.ProfilePlot(my_sphere, "radius", ["cell_mass"],
                      weight_field=None, accumulation=True)
p1d_acc.show()